In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from scipy.stats import spearmanr
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from shapely.geometry import Polygon, MultiPolygon
import matplotlib.patches as patches
import matplotlib.lines as mlines

In [ ]:
#==========================================USING THE SPEARMANS RANK CORRELATION COEFFICIENT==================================#

In [ ]:
A = pd.read_csv('GWS.csv')


In [ ]:
# Loaded data frames (replace 'data1.csv' and 'data2.csv' with your file paths)
df1 = pd.read_csv('GWS.csv')
df2 = pd.read_csv('recharge_with_coord.csv')
#df2.iloc[:, 2:] = df2.iloc[:, 2:] * 10 #tried to assume that the value of the recharge is in millimeter


In [ ]:
# Created an empty DataFrame to store the results
correlation_df = pd.DataFrame(columns=['latitude', 'longitude', 'correlation', 'p_value'])

In [ ]:
# Looped through each row and compute the correlation
for index, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    
    # Extracted time series from both data frames
    ts1 = row[2:]
    ts2 = df2.iloc[index, 2:]
    
    # Computed the Spearman rank correlation coefficient and p-value
    corr, p_value = spearmanr(ts1, ts2)
    
    # Appended the result to the correlation data frame
    correlation_df = correlation_df.append({'latitude': lat, 'longitude': lon, 'correlation': corr, 'p_value': p_value}, ignore_index=True)

# View edthe correlation_df
print(correlation_df)

In [ ]:
A = correlation_df['correlation'].idxmin()

In [ ]:
# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

cmap = mcolors.ListedColormap(['darkred', 'green', 'steelblue', 'yellow'])
bounds = [-1, -0.6, 0, 0.6, 1]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

correlation_df['color'] = np.select(
    [correlation_df['correlation'] <= -0.6,
     (correlation_df['correlation'] > -0.6) & (correlation_df['correlation'] < 0),
     (correlation_df['correlation'] >= 0) & (correlation_df['correlation'] < 0.6),
     correlation_df['correlation'] >= 0.6],
    ['darkred', 'green', 'steelblue', 'yellow'])

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(correlation_df['longitude'], correlation_df['latitude'], c=correlation_df['color'],
                marker='s', s=30, edgecolor=correlation_df['color'], transform=ccrs.PlateCarree())

# Created a ScalarMappable object for the colorbar
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # You need to set an empty array for the ScalarMappable object

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=[-0.8, -0.3, 0.3, 0.8], pad=0.05)
cbar.set_label('Correlation Coefficient', fontsize=15)
cbar.set_ticklabels(['< -0.6', '-0.6 to 0', '0 to 0.6', '> 0.6'])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Correlation between Groundwater Storage and Recharge \n GWS (without soil moisture-ERA5-Land)', fontsize=20, pad =20)
plt.show()

In [ ]:
corr_value = -0.83
correlation_df_subset = correlation_df[correlation_df['correlation'] < corr_value]
correlation_df_subset

In [ ]:
W = (correlation_df['correlation'] <= -0.6).sum()
X = ((correlation_df['correlation'] > -0.6) & (correlation_df['correlation'] < 0)).sum()
Y = ((correlation_df['correlation'] >= 0) & (correlation_df['correlation'] < 0.6)).sum()
Z = (correlation_df['correlation'] >= 0.6).sum()

print(W, X, Y, Z)

In [ ]:
data1 = df1.iloc[:, 3:]
data2 = df2.iloc[:, 3:]

# Transposed the DataFrames so that the time component becomes the index
data1 = data1.T
data2 = data2.T

# Converted the index of each DataFrame to a proper datetime format
data1.index = pd.to_datetime(data1.index)
data2.index = pd.to_datetime(data2.index)

# Created a figure object and add two axes to it
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

#Specified the axis
index = A

# Plotted the two time series on separate axes
ax1.plot(data1.iloc[:, index], label='GWS', color='b')
ax2.plot(data2.iloc[:, index], label='Recharge', color='r')

# Added axis labels and title
ax1.set_xlabel('Time')
ax1.set_ylabel('GWS (mm)', color='b')
ax2.set_ylabel('Recharge (mm)', color='r')
plt.title('Maximum negative correlation (-0.83) between GWS (without soil moisture) and recharge lat = 39.875 and lon = 20.375', fontsize=10)

# Added a legend and display the plot
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()


In [ ]:
#=============================Spearman's Rank Correlation where the soil is not considered====================================================

In [ ]:
# Loaded data frames (replace 'data1.csv' and 'data2.csv' with your file paths)
df1 = pd.read_csv('GWS_no_soil.csv')
df2 = pd.read_csv('recharge_with_coord.csv')
#df2.iloc[:, 2:] = df2.iloc[:, 2:] * 10 #tried to assume that the value of the recharge is in millimeter


In [ ]:
# Initialized a new data frame to store the correlation coefficients
correlation_df = pd.DataFrame(columns=['latitude', 'longitude', 'correlation', 'p_value'])

In [ ]:
# Looped through each row and compute the correlation
for index, row in df1.iterrows():
    lat = row['lat']
    lon = row['lon']
    
    # Extracted time series from both data frames
    ts1 = row[2:]
    ts2 = df2.iloc[index, 2:]
    
    # Computed the Spearman rank correlation coefficient and p-value
    corr, p_value = spearmanr(ts1, ts2)
    
    # Appended the result to the correlation data frame
    correlation_df = correlation_df.append({'latitude': lat, 'longitude': lon, 'correlation': corr, 'p_value': p_value}, ignore_index=True)

# Viewed the correlation_df
print(correlation_df)

In [ ]:
correlation_df

In [ ]:
A = correlation_df['correlation'].idxmax()
A

In [ ]:
# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

cmap = mcolors.ListedColormap(['darkred', 'green', 'steelblue', 'yellow'])
bounds = [-1, -0.6, 0, 0.6, 1]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

correlation_df['color'] = np.select(
    [correlation_df['correlation'] <= -0.6,
     (correlation_df['correlation'] > -0.6) & (correlation_df['correlation'] < 0),
     (correlation_df['correlation'] >= 0) & (correlation_df['correlation'] < 0.6),
     correlation_df['correlation'] >= 0.6],
    ['darkred', 'green', 'steelblue', 'yellow'])

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(correlation_df['longitude'], correlation_df['latitude'], c=correlation_df['color'],
                marker='s', s=30, edgecolor=correlation_df['color'], transform=ccrs.PlateCarree())

# Create a ScalarMappable object for the colorbar
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  # You need to set an empty array for the ScalarMappable object

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=[-0.8, -0.3, 0.3, 0.8], pad=0.05)
cbar.set_label('Correlation Coefficient', fontsize=15)
cbar.set_ticklabels(['< -0.6', '-0.6 to 0', '0 to 0.6', '> 0.6'])

ax.set_extent([-10, 49, 25.5, 49])

add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Spatial Correlation between Groundwater Storage and Recharge \n GWS (including soil moisture-ERA5-Land)', fontsize=20, pad =20)
plt.show()

In [ ]:
corr_value = 0.78
correlation_df_subset = correlation_df[correlation_df['correlation'] > corr_value]
correlation_df_subset

In [ ]:
W = (correlation_df['correlation'] <= -0.6).sum()
X = ((correlation_df['correlation'] > -0.6) & (correlation_df['correlation'] <= 0)).sum()
Y = ((correlation_df['correlation'] > 0) & (correlation_df['correlation'] < 0.6)).sum()
Z = (correlation_df['correlation'] >= 0.6).sum()

print(W, X, Y, Z)

In [ ]:
# Plotted the correlation coefficients on a map
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Added geographical features
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True, linestyle='--')

# Defined custom colormap and normalization function
cmap = mcolors.ListedColormap(['red', 'blue'])
norm = mcolors.BoundaryNorm([0, 0.05, 1], cmap.N)

# Determined the colors for each point based on the p_value column
colors = cmap(norm(correlation_df['p_value'].values))

# Plotted the correlation coefficients as scatter points with the same edgecolor as the points
sc = ax.scatter(correlation_df['longitude'], correlation_df['latitude'], c=correlation_df['p_value'],
                cmap=cmap, norm=norm, marker='o', s=100, edgecolor=colors, linewidth=1, transform=ccrs.PlateCarree())

# Added a colorbar
cbar = plt.colorbar(sc, orientation='horizontal', shrink=0.5, ticks=[0.025, 0.525])
cbar.set_label('P-value')
cbar.ax.set_xticklabels(['< 0.05', '> 0.05'])

# Set map extent (optional)
ax.set_extent([-11, 48, 27, 51])

plt.title('Level of Significance of Correlation between Groundwater Storage and Recharge')
plt.show()

In [ ]:
data1 = df1.iloc[:, 3:]
data2 = df2.iloc[:, 3:]

# Transposed the DataFrames so that the time component becomes the index
data1 = data1.T
data2 = data2.T

# Converted the index of each DataFrame to a proper datetime format
data1.index = pd.to_datetime(data1.index)
data2.index = pd.to_datetime(data2.index)

# Create a figure object and add two axes to it
fig, ax1 = plt.subplots(figsize=(10, 6))
ax2 = ax1.twinx()

#Specified the axis
index = A

# Plotted the two time series on separate axes
ax1.plot(data1.iloc[:, index], label='GWS', color='b')
ax2.plot(data2.iloc[:, index], label='Recharge', color='r')

# Added axis labels and title
ax1.set_xlabel('Time')
ax1.set_ylabel('GWS (mm)', color='b')
ax2.set_ylabel('Recharge (mm)', color='r')
plt.title('Maximum positive correlation (0.78) between GWS (Including soil moisture) and recharge \n lat = 44.125 and lon = -0.375', fontsize=10)

# Added a legend and display the plot
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()
